## Question 1
##### Use the BeautifulSoup package or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe

Importing the request library

In [106]:
import requests

URL of  Wikipedia page(for Toronto neighbourhood)

In [107]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)

In [108]:
page.status_code

200

### Installing and importing beautifulsoup4

In [109]:
#!pip install beautifulsoup4

In [110]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [111]:
results = soup.find(id='mw-content-text')

In [112]:
#print(results.prettify())

Taking only the < table > tag into consideration

In [113]:
table_cont = results.find('table', class_='wikitable sortable')
#print(table_cont)

In [114]:
len(table_cont)

2

Creating a list which contains the data of all three coloumns

In [115]:
li = []
for row in table_cont.find_all('tr'):
    for cell in row.find_all('td'):
        #print(cell.text,end ='')
        li.append(cell.text)

Removing the \n tag in each value in li list

In [116]:
li2 = []
for i in range(0,len(li)):
    li2.append(li[i].replace('\n',''))

In [117]:
print(len(li2))

540


Importing pandas to read the dataframe

In [118]:
import pandas as pd

Creating the dataframe

In [119]:
column_names = ['PostalCode','Borough','Neighborhood']
neighborhoods = pd.DataFrame(columns=column_names)

Filling the values

In [120]:
i=0
while i<=538:
    neighborhoods = neighborhoods.append({'PostalCode': li2[i],
                                          'Borough': li2[i+1],
                                          'Neighborhood': li2[i+2]}, ignore_index=True)
    i = i+3

In [121]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Dropping all the values which is not assigned

In [122]:
neighborhoods.drop(neighborhoods[neighborhoods['Borough'] == 'Not assigned'].index, inplace = True) 

In [123]:
neighborhoods.drop(neighborhoods[neighborhoods['Neighborhood'] == 'Not assigned'].index, inplace = True) 

In [124]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [125]:
neighborhoods.reset_index()

,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...,...
98,160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,165,M4Y,Downtown Toronto,Church and Wellesley
100,168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [126]:
neighborhoods.shape

(103, 3)

## Question 2

#### Use the Geocoder package or the csv file to create dataframe with longitude and latitude values

I will be using the csv file to create the dataframe with longitude and latitude values

In [128]:
import os
os.chdir('E:\DSc\Final project\week 3')

In [129]:
goedata_df = pd.read_csv('Geospatial_Coordinates.csv')

In [130]:
goedata_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [137]:
Toronto_df = pd.merge(neighborhoods, goedata_df, left_on='PostalCode', right_on='Postal Code')

In [138]:
Toronto_df.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,43.662301,-79.389494


In [142]:
Toronto_df.drop(columns='Postal Code' ,inplace=True)

In [144]:
Toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
